In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
import redis
import json

In [2]:
def get_fills_from_redis(strat = 'capital_txo_main', acc='TAIFEX100',date=dt.date.today(),night_session=True, redishost='prod1.capital.radiant-knight.com'):
    r = redis.StrictRedis(host=redishost, port=6379,db=0)
    rk = '{}:{}'.format(acc, date.strftime('%Y%m%d'))
    rk_strat = '{}:{}'.format(strat, date.strftime('%Y%m%d')) 
#     rk = '*{}*'.format(acc)
    if night_session:
        rk = rk + 'E'
        rk_strat = rk_strat + 'E'
    if rk not in [k.decode() for k in r.keys()]:
        print('Key:{} not exists!'.format(rk))
        if rk_strat in [k.decode() for k in r.keys()]: 
            rk = rk_strat
        else:
            return None
    msgs = r.lrange(rk, 0, -1)
#     print(msgs)
    trd_df = pd.DataFrame([json.loads(m.decode()) for m in msgs])
    if trd_df.empty:
        print('Empty redis key')
        return None
    if trd_df['ts'][0] > 1e11:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x/1e6))
    else:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x))
    trd_df.set_index('time',inplace=True)

    return trd_df

In [3]:
tmf0304=get_fills_from_redis('capital_electron_tmf', date=dt.date(2025,3,4), redishost='prod1.capital.radiant-knight.com', night_session=True)

Key:TAIFEX100:20250304E not exists!


In [4]:
pd.set_option('display.max_rows',None)

In [7]:
tmf0304.between_time('02:00:01','05:30:05')[['instr','px','sz','alqortake']]

,instr,px,sz,alqortake
time,,,,
2025-03-04 02:05:07.281488,FUT_TAIFEX_TMF:202504,22528,-2,take
2025-03-04 02:05:07.281520,FUT_TAIFEX_TXF:202504,22521,1,take
2025-03-04 02:05:07.281533,FUT_TAIFEX_TMF:202504,22527,-1,take
2025-03-04 02:05:07.281540,FUT_TAIFEX_TMF:202504,22526,-2,take
2025-03-04 02:05:07.281548,FUT_TAIFEX_TMF:202504,22525,-14,take
2025-03-04 02:07:30.789965,FUT_TAIFEX_TMF:202504,22505,-5,take
2025-03-04 02:07:30.790004,FUT_TAIFEX_TXF:202504,22497,1,take
2025-03-04 02:07:30.790018,FUT_TAIFEX_TMF:202504,22503,-1,take
2025-03-04 02:07:30.790026,FUT_TAIFEX_TMF:202504,22502,-2,take
